In [1]:
#For preprocessing of data- preprocessing pipeline

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math
import num2words

nltk.download("popular")
!pip install num2words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words



def remove_stop_words(data):
    stop_words = stopwords.words('french')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

def convert_lower_case(data):
    return np.char.lower(str(data))

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")


def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/ashar/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/ashar/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/ashar/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/ashar/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/ashar/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/ashar/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!

In [3]:
#Extract all title and description from database

fdata=pd.read_csv('/home/ashar/fiverr/search engine/database.csv',encoding = "ISO-8859-1")
main_data=fdata

for index,row in main_data.iterrows():
    main_data['full']=main_data['Poste du tiers_ Title of first name']+'  '+main_data['Compétences(details about title of first name to take into account)']
print(main_data)

     Nom du tiers_First name  \
0                         Ya   
1                         Ma   
2                         Ma   
3                         Da   
4                         Da   
...                      ...   
7528                      ST   
7529                      LU   
7530                      DA   
7531                      SO   
7532                      CO   

                    Poste du tiers_ Title of first name  \
0                  Consultant IT indépendant, PMP, ITIL   
1                        Consultant indépendant, we IT.   
2                                  Freelance IT Manager   
3                               Consultant IT Freelance   
4     Finance Consultant in Digital Transformation a...   
...                                                 ...   
7528  Responsable Informatique - Technologies Libres...   
7529                                Coach professionnel   
7530         Business Analyst / Consultant informatique   
7531              Directe

In [4]:
#Does preprocessing of data with above mentioned preprocess pipeline

processed_text=[]
i=0
for t in main_data['full']:
    processed_text.append(word_tokenize(str(preprocess(t))))
    if i%100==0:
        print("text: "+str(i))
    i+=1

print(len(processed_text))

text: 0
text: 100
text: 200
text: 300
text: 400
text: 500
text: 600
text: 700
text: 800
text: 900
text: 1000
text: 1100
text: 1200
text: 1300
text: 1400
text: 1500
text: 1600
text: 1700
text: 1800
text: 1900
text: 2000
text: 2100
text: 2200
text: 2300
text: 2400
text: 2500
text: 2600
text: 2700
text: 2800
text: 2900
text: 3000
text: 3100
text: 3200
text: 3300
text: 3400
text: 3500
text: 3600
text: 3700
text: 3800
text: 3900
text: 4000
text: 4100
text: 4200
text: 4300
text: 4400
text: 4500
text: 4600
text: 4700
text: 4800
text: 4900
text: 5000
text: 5100
text: 5200
text: 5300
text: 5400
text: 5500
text: 5600
text: 5700
text: 5800
text: 5900
text: 6000
text: 6100
text: 6200
text: 6300
text: 6400
text: 6500
text: 6600
text: 6700
text: 6800
text: 6900
text: 7000
text: 7100
text: 7200
text: 7300
text: 7400
text: 7500
7533


In [5]:
#Extracting vocabulory from every extracted data

N=len(processed_text)
DF = {}

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

In [6]:
#store vocabolary size
total_vocab_size = len(DF)
total_vocab_size

8167

In [7]:
#store total vocabolary in list
total_vocab = [x for x in DF]
total_vocab[:20]

['consult',
 'it',
 'indépend',
 'pmp',
 'itil',
 'scrum',
 'softwar',
 'develop',
 'project',
 'manag',
 'développ',
 'logiciel',
 'chang',
 'gestion',
 'projet',
 'we',
 'stratégi',
 'gouvern',
 'commerci',
 'servic']

In [8]:
#find document frequency of word
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [9]:
#find tf-idf of every word
from collections import Counter
doc = 0

tf_idf = {}

for i in range(N):
    
    tokens = processed_text[i]
    
    counter = Counter(tokens)
    words_count = len(tokens)
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        
        tf_idf[doc, token] = tf*idf

    doc += 1
    
len(tf_idf)

216015

In [10]:
#find cosine similarity of sentences
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

len(tf_idf)

216015

In [11]:
#indexing of words
D = np.zeros((N, total_vocab_size))
cnt=0
for i in tf_idf:
    if cnt%100000==0:
        print(cnt)
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass
    cnt+=1

0
100000
200000


In [12]:
#generate vector for words
def gen_vector(tokens):

    Q = np.zeros((len(total_vocab)))
    
    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log((N+1)/(df+1))

        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [13]:
#find cosine similarity

def cosine_similarity(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    d_cosines = []
    
    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        
    out = np.array(d_cosines).argsort()[-k:][::-1]

    return out

In [17]:
from gensim import corpora, models, similarities
import jieba
import re
#sentence similarity
def isQualified(sentence,sentList,simScore):
    maxVal=0
    texts = sentList
    keyword = sentence
    texts = [jieba.lcut(str(text)) for text in texts]
    dictionary = corpora.Dictionary(texts)
    feature_cnt = len(dictionary.token2id)
    corpus = [dictionary.doc2bow(text) for text in texts]
    tfidf = models.TfidfModel(corpus) 
    kw_vector = dictionary.doc2bow(jieba.lcut(keyword))
    index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = feature_cnt)
    sim = index[tfidf[kw_vector]]
    
    return sim

querydata=pd.read_csv('updatedq.csv',encoding = "ISO-8859-1")
# print(querydata)
results=pd.DataFrame(columns=['query','top_names','top_titles','top_descriptions','score'])
querycount=0

query='Ingénieur Etudes et Développement Backend PHP '#type query here
keywords='Stack, java stack, java angular'.split(',') #give coma separated keywords string
description='' #give description of query
total_query=query+description
top_titles=[]
top_des=[]
top_full=[]
top_names=[]
extract_titles=[]
extract_des=[]
extract_names=[]
Q = cosine_similarity(50, total_query)
#check for keywords
for x in range(len(Q)):
    title=str(main_data.iloc[Q[x]]['Poste du tiers_ Title of first name'])
    full=str(main_data.iloc[Q[x]]['full'])
    for word in keywords:
        if word in full:   #for title bias change 'full' to 'title', 'full' is for description biasness
            top_titles.append(main_data.iloc[Q[x]]['Poste du tiers_ Title of first name'])
            top_des.append(main_data.iloc[Q[x]]['Compétences(details about title of first name to take into account)'])
            top_full.append(main_data.iloc[Q[x]]['full'])
            top_names.append(main_data.iloc[Q[x]]['Nom du tiers_First name'])
            break

scores=[]
for text in top_full:
    score=isQualified(query,[text,'sample text'],0)[0]
    scores.append(score)

cols=[]
cols_count=0
for s in scores:
    cols_count+=1
    cols.append(cols_count)

if len(cols)>0:
    max_score=max(scores)
    scoresdf=pd.DataFrame(columns=cols)

    scoresdf.loc[len(scoresdf)]=scores


    nlargest = 5
    topnlocs = np.argsort(-scoresdf.values, axis=1)[:, 0:nlargest]

    for locs in topnlocs[0]:
        extract_titles.append(top_titles[int(locs)])
        extract_des.append(top_des[int(locs)])
        extract_names.append(top_names[int(locs)])

    results.loc[len(results)]=[query,extract_names,extract_titles,extract_des,max_score]
            
print(results)

Empty DataFrame
Columns: [query, top_names, top_titles, top_descriptions, score]
Index: []


In [18]:
#To convert dataframe into json
import json
df_json=results.to_json(orient="split")
print(df_json)

{"columns":["query","top_names","top_titles","top_descriptions","score"],"index":[],"data":[]}


In [ ]:
## code to convert dataframe into excel

import nltk
import pandas as pd
from odf import text, teletype
from odf.opendocument import load
from openpyxl import load_workbook

def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,truncate_sheet=False,**to_excel_kwargs):

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    # Python 2.x: define [FileNotFoundError] exception if it doesn't exist 
    try:
        FileNotFoundError
    except NameError:
        FileNotFoundError = IOError


    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)

        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)

        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()
    
append_df_to_excel('PATH_TO_SAVE_THE_FILE/results_biased_description.xlsx',results)